In [3]:
import speech_recognition as sr

recognizer = sr.Recognizer()

with sr.Microphone() as source:
    print("🎤 Nói đi...")
    recognizer.adjust_for_ambient_noise(source)
    audio = recognizer.listen(source)

try:
    text = recognizer.recognize_google(audio, language="vi-VN")
    print("📝 Bạn nói:", text)
except sr.UnknownValueError:
    print("❌ Không nhận diện được")
except sr.RequestError as e:
    print("❌ Lỗi dịch vụ:", e)


🎤 Nói đi...
📝 Bạn nói: alo 1 2 3 4 5 6 9 10


In [10]:
from google.cloud import translate_v2 as translate

translate_client = translate.Client()

text = "¡Hola amigos y amigas!"
target_language = "en"

result = translate_client.translate(text, target_language=target_language)

print("Input:", result['input'])
print("Translated:", result['translatedText'])


ImportError: cannot import name 'translate_v2' from 'google.cloud' (unknown location)

<div align="center">
  <img src="./image/fullpipeline.png" alt="Full Pipeline" style="width:100%; max-width:900px;" />
</div>

<div align="center">

![AI Agent](./image/readme/aiagent.jpg)

</div>

<p align="center">
  <img src="./image/readme/gemini.jpg" width="200" alt="Gemini"/>
  <img src="./image/readme/react.jpg" width="200" alt="React"/>
  <img src="./image/readme/duckduckgo.png" width="200" alt="DuckDuckGo"/>
  <img src="./image/readme/langchain.jpg" width="200" alt="LangChain"/>
</p>

---

# 🤖 AI Agent - Trợ Lý Thông Minh Đa Năng

## 📌 Giới Thiệu

> **⚠️ Lưu ý:** Repo này được tạo ra để **test và xài thử các API** đồng thời **thử vibe coding một con AI Agent đơn giản**.
> 
> Trong tương lai sẽ sử dụng thêm nhiều API để test và build trong lúc thời gian rảnh rỗi, nên đừng quá khắt khe nhé! 😊
> 
>Repository này dùng test và khám phá thêm các chức năng nên nó **không nằm trong 1 phạm vi cụ thể**. Chỉ đơn giản là **khám phá những thứ tò mò chưa làm bao giờ thôi**.
>
> **⭐ Nếu thấy hay thì hãy ủng hộ bằng cách cho repo một Star nhé!**

<div align="center">

![Demo](./output.gif)

</div>
AI Agent mạnh mẽ với khả năng tìm kiếm web, tính toán toán học, và phân tích dữ liệu, sử dụng Gemini API làm LLM chính.




## Full Pipeline

<div align="center">
  <img src="./image/fullpipeline.png" alt="Full Pipeline" style="width:100%; max-width:900px;" />
</div>

- **1. Tầng giao diện người dùng (User Interface):** Hỗ trợ nhiều hình thức đầu vào: văn bản, hình ảnh, âm thanh. Đầu ra hiển thị đa dạng: văn bản, hình ảnh, video, biểu đồ, âm thanh. Có tích hợp `ngrok` để tunneling, nâng cao bảo mật và khả năng truy cập từ xa.

- **2. Tầng xử lý chính (Core Framework & Backend):** Sử dụng `FastAPI` làm framework chính, kèm `Pydantic` để validate dữ liệu. API server tiếp nhận và xử lý yêu cầu. `Smart Router` / Decision Maker định tuyến yêu cầu đến dịch vụ AI phù hợp.

- **3. Tầng AI & Xử lý chuyên sâu:** Hệ thống tích hợp nhiều công cụ AI theo lĩnh vực:
  - **LLM & Logic:** Google Gemini, Local LLM (Transformers, Ollama, ...)
  - **Xử lý hình ảnh (Vision AI):** Salesforce BLIP, OCR (EasyOCR / PaddleOCR)
  - **Xử lý âm thanh:** Speech-to-Text (Whisper), Text-to-Speech (gTTS)
  - **Tìm kiếm & tri thức:** DuckDuckGo, SerpAPI, WolframAlpha
  - **Phân tích dữ liệu:** Pandas, NumPy, Matplotlib, Plotly
  - **Tạo phương tiện đa phương tiện:** Text-to-Image (Clipdrop), Text-to-Video, Slide/Presentation generator

## 🐳 Cài Đặt & Chạy với Docker

> **💡 Khuyến nghị:** Sử dụng Docker để triển khai nhanh chóng, tránh các vấn đề về môi trường và dependencies!
